# INSIGHTS


Usando a Regressão linear simples notei que há um resultado razoavel quanto ao vies.

Usando a Random Forest notei que o R2 é inferior ao da RL, e há uma varincia maior e em vários momentos os valores são muito proximos ao valor real.

De modo geral avaliando o timeframe D1 o modelo de regressão esta se demonstrando favoravel a questão de indicar a direção do mercado.

Utilizando o exemplo do Andre (tera) há uma expectativa de melhoria uma vez que ele utilizou boas features (LTA/LTB, Suporte e resistencia, Média, Bollinger e RSI). O modelo em questão foi colocado em produção com uma acurácia de 75%.

<b>Alertas!</b><p>
quando o Close for igual ao Open ....deixar um feedback de DOJI<p>
para plotagem ou modelagem considerar o o drop_duplicates(subset=time, keep='last')

# Problemas

as metricas estão ok ... porém o quanto performa em tempo real?

o velho problema de que a previsão é referente a uma comparação ao open ... e talvez no momento do predict já tenhamos perdido o timming

#### Bibliotecas

In [1]:
# BASIC
import pandas as pd
import numpy as np
import datetime as dt


# CLASSIFIER
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb

#REGRESSOR
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# PRE-PROCESSING
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#METRICS
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score


# STATISTIC
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# DATA VIZ
import matplotlib.pyplot as plt
import seaborn as sns

# WARNING
import warnings
warnings.filterwarnings("ignore")

### Load Data

In [2]:
# APPL HORA EXEMPLO DO ANDRE KUNIYOSHI
colunas = ['Time', 'Open', 'High', 'Low', 'Close','Adj Close','Volume']
df_base = pd.read_csv(r'C:\_PY\Trade_NB\_DS\Bases\df_AAPL_hora.csv')
df_base.columns = colunas
df_base['Time'] = pd.to_datetime(df_base['Time'], utc=True)
df_base.set_index('Time', inplace=True)

df_base = df_base[['Open', 'High', 'Low', 'Close','Adj Close','Volume']]

# conferindo o tamanho do dataset
print('AAPL tamanho:', df_base.shape)

AAPL tamanho: (2273, 6)


### OUTLIERS

### Serie Estacionaria

In [3]:
# Agora, vamos analisar se a série é estacionária, ou não. Para que seja, ela precisa ter a média e a variância constantes no tempo, além de ter correlação entre 2 períodos constante

# Vamos utilizar o teste de Dickey Fuller para verificar se nossa série é estacionária ou não.
# H0 -> Série não estacionária (p-value > 0.05)
# H1 -> Série estacionária (p-value <= 0.05)

# o resultado estatístico mostrou que a série não é estacionária. (p-value > 0.05)
#Então, vamos utilizar a técnica de diferenciação para tornar a série estacionária, para podermos aplicar o modelo ARIMA

adfuller_aapl_hora = adfuller(df_base['Adj Close'])
print('ADF Statistics:', adfuller_aapl_hora[0])
print('p-value :', adfuller_aapl_hora[1])

ADF Statistics: -1.237289366571211
p-value : 0.6573526804673975


In [4]:
# Calculo do retorno percentual (1º diferenciação)
df_arima = df_base.copy()
df_arima['return_percent'] = df_arima['Close'].pct_change()
df_arima.dropna(inplace=True)

In [5]:
# rodando o método de Dickey Fuller para retorno PERCENTUAL
adfuller_df_arima = adfuller(df_arima['return_percent'])
print('ADF Statistics AAPL HORA:', adfuller_df_arima[0])
print('p-value AAPL HORA:', adfuller_df_arima[1])

ADF Statistics AAPL HORA: -8.855398205628859
p-value AAPL HORA: 1.532812259092094e-14


In [6]:
# TIMEFRAME
#BASE MINUTOS DIARIO 510 
#BASE HORAS POR PREGAO DIARIO: 7
#BASE MESES PREGAO ANO: 12

time_frame = 7

In [7]:
# https://maxwellpaparelli.medium.com/time-series-s%C3%A9ries-temporais-com-python-f4e74fd45b0b
df_log = np.log(df_base)
ma_log = df_log.rolling(time_frame).mean()

#subtrair média do log dos dados, iremos fazer a média em 12, pois iremos fazer anualmente
df_sub = (df_log - ma_log).dropna()
ma_sub = df_sub.rolling(time_frame).mean()
#desvio padrão
std_sub = df_sub.rolling(time_frame).std()

In [8]:
adfuller_df_arima = adfuller(df_log['Adj Close'])
print('ADF Statistics AAPL HORA:', adfuller_df_arima[0])
print('p-value AAPL HORA:', adfuller_df_arima[1])

ADF Statistics AAPL HORA: -1.1891641156083415
p-value AAPL HORA: 0.6782395798917242


In [9]:
#repetir o ADF
X_sub = df_sub['Adj Close'].values

# aplicar ADF e imprimir o resultado
result_sub = adfuller(X_sub)
print('Dickey-Fuller Aumentado')
print('Teste Estatístico: {:.4f}'.format(result_sub[0]))
print('Valor-p: {:.10f}'.format(result_sub[1]))
print('Valores Críticos:')
for key, value in result_sub[4].items():
	print('\t{}: {:.4f}'.format(key, value))

Dickey-Fuller Aumentado
Teste Estatístico: -8.2795
Valor-p: 0.0000000000
Valores Críticos:
	1%: -3.4333
	5%: -2.8628
	10%: -2.5675


In [10]:
#Diferenciação

#aplicar diferenciação
df_diff = df_sub.diff(1)
ma_diff = df_diff.rolling(time_frame).mean()
#desvio padrão
std_diff = df_diff.rolling(time_frame).std()

#plotar a diferenciação
fig, ax = plt.subplots()
df_diff.plot(ax=ax, legend=False)
ma_diff.plot(ax=ax, legend=False, color='r')
std_diff.plot(ax=ax, legend=False, color='g')
plt.tight_layout()

#extrair apenas os valores e retirar os valores NA
X = df_diff['Adj Close'].dropna().values

# aplicar ADF e imprimir o resultado
result_diff = adfuller(X)
print('Dickey-Fuller Aumentado')
print('Teste Estatístico: {:.4f}'.format(result_diff[0]))
print('Valor-p: {:.10f}'.format(result_diff[1]))
print('Valores Críticos:')
for key, value in result_sub[4].items():
	print('\t{}: {:.4f}'.format(key, value))

Dickey-Fuller Aumentado
Teste Estatístico: -15.7205
Valor-p: 0.0000000000
Valores Críticos:
	1%: -3.4333
	5%: -2.8628
	10%: -2.5675


In [11]:
df_diff.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2259 entries, 2021-01-04 20:30:00+00:00 to 2022-04-18 19:30:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2258 non-null   float64
 1   High       2258 non-null   float64
 2   Low        2258 non-null   float64
 3   Close      2258 non-null   float64
 4   Adj Close  2258 non-null   float64
 5   Volume     2258 non-null   float64
dtypes: float64(6)
memory usage: 123.5 KB


In [12]:
df_diff = df_diff.dropna()

In [13]:
df_diff['return_percent'] = df_diff['Close'].pct_change()

### Teste de Normalidade

In [14]:
#Agora, podemos verificar se as distribuições de retorno são normais ou não.

#Para isso, vamos utilizar o método de shapiro, da lib scipy stats
print('Dataset HORAS, retorno PERCENTUAL (1º dif.):', stats.shapiro(df_diff['return_percent']))

Dataset HORAS, retorno PERCENTUAL (1º dif.): ShapiroResult(statistic=nan, pvalue=1.0)


In [15]:
df = df_diff.copy()

In [16]:
df['amplitude'] = df['High'] - df['Low']
df['body'] = np.where(df["Close"] > df["Open"], df["Close"] - df["Open"], df["Open"] - df["Close"])
df['pavil'] = df['amplitude'] - df['body']
df['pavil_call'] = np.where(df["Close"] > df["Open"], df['High'] - df['Close'], df['High'] - df['Open'])
df['pavil_put'] = np.where(df["Close"] > df["Open"], df['Open'] - df['Low'], df['Close'] - df['Low'])
df["retorno_volume"] = df["Volume"].pct_change(1)*100
df["retorno"] = df["Close"].pct_change(1)*100

In [17]:
#-----------------------------------FEATURE ENGINEERING--------------------------------------------

In [18]:
# CRIANDO FEATURE RSI
def criar_rsi(df):
    n = 20
    def rma(x, n, y0):
        a = (n-1) / n
        ak = a**np.arange(len(x)-1, -1, -1)
        return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]

    df['change'] = df['Adj Close'].diff()
    df['gain'] = df.change.mask(df.change < 0, 0.0)
    df['loss'] = -df.change.mask(df.change > 0, -0.0)
    df['avg_gain'] = rma(df.gain[n+1:].to_numpy(), n, np.nansum(df.gain.to_numpy()[:n+1])/n)
    df['avg_loss'] = rma(df.loss[n+1:].to_numpy(), n, np.nansum(df.loss.to_numpy()[:n+1])/n)
    df['rs'] = df.avg_gain / df.avg_loss
    df['rsi'] = 100 - (100 / (1 + df.rs))
    return df

In [19]:
# CRIANDO FEATURE BOLLINGER BAND
def criar_bollinger(df):
  # calculando a média móvel e limites superior e inferiror
  # limites com base em 2 desvios padrão
  mid = df['Adj Close'].rolling(20).mean()
  std = df['Adj Close'].rolling(20).std()
  up = mid + std
  low = mid - std

  # criando features para a média e os limites
  df['upper'] = up
  df['mid'] = mid
  df['low'] = low
  df['bbp'] = (df['Adj Close'] - df['low'])/(df['upper'] - df['low'])
  df.dropna(inplace=True)
  return df

In [20]:
# RESISTÊNCIA
def is_resistance(df,i):
  resistance = (df['High'][i] > df['High'][i-1]
                and df['High'][i] > df['High'][i+1]
                and df['High'][i+1] > df['High'][i+2]
                and df['High'][i-1] > df['High'][i-2])
  return resistance

# SUPORTE
def is_support(df,i):
  support = (df['Low'][i] < df['Low'][i-1]
             and df['Low'][i] < df['Low'][i+1]
             and df['Low'][i+1] < df['Low'][i+2]
             and df['Low'][i-1] < df['Low'][i-2])
  return support

def suporte_resistencia(df):
  # resistência verdadeiro -> 1 (vender)
  # suporte verdadeiro -> 0 (comprar)
  # outros (2)

  # criando feature com valores 2
  df['suport_resistencia'] = 2

  # definindo os valores 1 e 0
  for i in range(2, df.shape[0] - 2):
    if is_resistance(df,i):
      df['suport_resistencia'][i] = 1 # definindo 1 para resistência
    elif is_support(df,i):
      df['suport_resistencia'][i] = 0 # definindo 0 para suporte
  return df

In [21]:
# LTA E LTB
# 0.5 são classificadas como tendência de alta, menos que -0.5 são tendência de baixa
def lta_ltb(df):
  df2 = df.reset_index()
  df['corr'] = (df2['Adj Close'].rolling(20).corr(pd.Series(df2.index))).tolist()
  df.dropna(inplace=True)

  def condition(x):
      if x<=-0.5:
          return -1
      elif x>-0.5 and x<0.5:
          return 0
      else:
          return 1
  df['corr_class'] = df['corr'].apply(condition)

  return df

In [22]:
# MÉDIA MÓVEL
def media_movel(df, coluna, defasagem):
  df['media_movel'] = df[coluna].rolling(20).mean()
  return df

In [23]:
# FEATURES DE TEMPO
def feat_temporais(df):
  df['dia_semana'] = df.index.dayofweek
  df['horario'] = df.index.hour
  df['mes'] = df.index.month
  return df

In [24]:
# CRIANDO A TARGET
def target(df):

  # criando feature com 1h de defasagem (com hora anterior)
  df['def_1'] = df['Adj Close'].shift(1)
  # criando feature comparando valor atual com o defasado
  df['subt'] = df['Adj Close'] - df['def_1']
#  criando a target de subida ou descida do valor da ação
#  0 -> caiu (com relação ao anterior)
#  1 -> subiu (com relação ao anterior)
#  2 -> igual ao anterior
  
  df['target'] = df['subt'].apply(lambda x: int(0) if x<0 else int(1) if x>0 else int(2))
  #df['target'] = df['subt'].apply(lambda x: int(0) if x<=0 else int(1))
  return df


In [25]:
# FEATURES DEFASADAS 
def constroi_features_defasadas(df,lista_features,defasagem_maxima):
    # Constrói features defasadas com base na base original
    # Copia a base
    df_cop = df.copy()
    for feat in lista_features:       
        for i in range(1,defasagem_maxima+1):
            df_cop[str(feat)+'_def_'+str(i)] = df_cop[feat].shift(i)
    
    df_cop.dropna(inplace=True)
    return df_cop

In [26]:
# FEATURES FUTURAS
def constroi_features_futuras(df,feature,defasagem):
    # Constrói features defasadas com base na base original
    # Copia a base
    df_cop = df.copy()

    df_cop[str(feature)+'_fut'] = df_cop[feature].shift(-defasagem)
    return df_cop

In [27]:
df = criar_rsi(df) # 'change', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs', 'rsi'
df = criar_bollinger(df) #'upper', 'mid', 'low','bbp'
df = suporte_resistencia(df) #'suport_resistencia'
df = lta_ltb(df) # 'corr', 'corr_class'
df = media_movel(df, 'Adj Close', 20) # 'media_movel'
df = feat_temporais(df) # 'dia_semana', 'horario', 'mes'

In [28]:
###-----------------------------------FUNÇÃO DO MODELO-------------------------------------------- '''

In [29]:
def modelo(df, target_):
    X_test = df.drop(target_, axis=1)[-1:]

    X_train = df[:-1].dropna().drop(target_, axis=1)
    y_train = df[:-1].dropna()[target_]

    xgb = XGBClassifier(random_state=42,
			gamma = 0.1,
			max_depth = 8,
			n_estimators = 100,
			n_jobs=-1)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    y_proba = xgb.predict_proba(X_test)
    y_proba = y_proba[:, 1]
    return y_pred, y_proba

In [30]:
def preprocessamento(base, corte_treino_teste,target):
  # separando a base da target
  X = base.drop(target, axis=1)
  y = base[target]
  # cortando em treino e teste
  X_train = X[X.index<=corte_treino_teste]
  X_test = X[X.index>corte_treino_teste]
  y_train = y[y.index<=corte_treino_teste]
  y_test = y[y.index>corte_treino_teste]
  return X_train, X_test, y_train, y_test

In [31]:
modelos_candidatos = {'Árvore':DecisionTreeClassifier(random_state=42),
                      'RandomForest':RandomForestClassifier(random_state=42),
                      'XGBoost':XGBClassifier(random_state=42)
                     }
def avaliacao_modelos(modelos,X_train,y_train):
  avaliacao = {}
  scores = []
  ts = TimeSeriesSplit(n_splits=5).split(X_train,y_train)
  for nome,model in modelos_candidatos.items():
    # Avalia a crossvalidação
    score = cross_val_score(model, # Escolhendo o nosso modelo da vez
                          X_train, y_train, # Nossos dados, excluindo o teste
                          cv=TimeSeriesSplit(n_splits=5).split(X_train,y_train), # Validação cruzada temporal
                          scoring='accuracy', # Usando a acurácia como métrica
                          ).mean() # Tirando a média de todos os folds
    avaliacao[nome] = score
    scores.append(score)
  
  return avaliacao, scores

In [32]:
##-----------------------------------CRIANDO DATASET-------------------------------------------- '''
hora_previsao = 1
df = target(df)
df.dropna(inplace=True) #, 'bbp', 'corr_class'
df = df[['bbp', 'corr_class','amplitude','body','pavil_call', 'pavil_put', 'retorno_volume', 'retorno','target', 'Adj Close', 'Volume', 'rsi', 'suport_resistencia', 'media_movel', 'dia_semana', 'horario', 'mes']]
#df = df[['bbp', 'corr_class','target', 'Adj Close', 'Volume', 'rsi', 'suport_resistencia', 'media_movel', 'dia_semana', 'horario', 'mes']]
df = constroi_features_defasadas(df,['Adj Close'],20)
df = constroi_features_futuras(df,'target',hora_previsao)
df_model = df.drop('target', axis=1)

In [33]:
df = df.dropna()

In [34]:
#, 'bbp','corr_class'
#'amplitude','body', 'pavil_call', 'pavil_put', 'retorno_volume','retorno'
features = df.loc[:,['amplitude','body', 'pavil_call', 'pavil_put', 'retorno_volume','retorno','bbp','corr_class', 'target', 'Adj Close', 'Volume', 'rsi', 'suport_resistencia', 'media_movel', 'dia_semana',
       'horario', 'mes', 'Adj Close_def_1', 'Adj Close_def_2',
       'Adj Close_def_3', 'Adj Close_def_4', 'Adj Close_def_5',
       'Adj Close_def_6', 'Adj Close_def_7', 'Adj Close_def_8',
       'Adj Close_def_9', 'Adj Close_def_10', 'Adj Close_def_11',
       'Adj Close_def_12', 'Adj Close_def_13', 'Adj Close_def_14',
       'Adj Close_def_15', 'Adj Close_def_16', 'Adj Close_def_17',
       'Adj Close_def_18', 'Adj Close_def_19', 'Adj Close_def_20']]
y = df['target_fut']

### Normalização

In [35]:
scaler = MinMaxScaler()#StandardScaler() #MinMaxScaler()

### Treino Teste

In [36]:
df = df.dropna()

In [37]:
x_train, x_test, y_train, y_test = train_test_split(features, y, test_size = 0.2)

In [38]:
x_train = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)

### naive bayes

In [39]:
modelo = GaussianNB()
modelo.fit(x_train, y_train)
previsoes = modelo.predict(x_test)
print('\nClassification Report\n', metrics.classification_report(y_test, previsoes)) 
# sem padronização: 0.55
# com padronização: 0.54
# com nomalização: 0.50

# EURUSD - Após Serie Estacionaria log ~2863 registros
# sem padronização: 48
# com padronização: 70
# com nomalização: 75 - med 510: 73

# AAPL ~400 registros
# sem padronização: 51
# com padronização: 35 
# com nomalização: 37 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 79 com todas as features 71
# com padronização: 71 removendo os outliers: 68 | com todas as features 64
# com nomalização: 74  com todas as features: 68  


Classification Report
               precision    recall  f1-score   support

         0.0       0.83      0.40      0.54       225
         1.0       0.59      0.91      0.72       211

    accuracy                           0.65       436
   macro avg       0.71      0.66      0.63       436
weighted avg       0.71      0.65      0.63       436



### Logistic Regression

In [ ]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

y_pred = pd.Series(classifier.predict(x_test))
y_test = y_test.reset_index(drop=True)

print('\nClassification Report\n', metrics.classification_report(y_test, y_pred)) 
# sem padronização: 0.55
# com padronização: 0.54
# com nomalização: 0.53

# EURUSD - Após Serie Estacionaria log
# sem padronização: 75
# com padronização: 76
# com nomalização: 75 - med 510: 75

# AAPL
# sem padronização: 48
# com padronização: 48 
# com nomalização: 52 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 74 com todas as features 66
# com padronização: 75 removendo os outliers: 68 | com todas as features 78
# com nomalização: 76 com todas as features 75  

### Random Forest Classifiers

In [ ]:
# importar o modelo Random Forest Regressor

# treinando o modelo 
classifier = RandomForestClassifier(n_estimators= 21, random_state=42).fit(x_train, y_train);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions3_labels = classifier.predict(x_test)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
p = pd.DataFrame({'Real': y_test, 'Previsto': predictions3_labels})  
print('\nClassification Report\n', metrics.classification_report(y_test, predictions3_labels)) 
# sem padronização: 0.56
# com padronização: 0.57
# com nomalização: 0.56

# EURUSD - Após Serie Estacionaria log
# sem padronização: 75
# com padronização: 75
# com nomalização: 74 - med 510: 74


# AAPL
# sem padronização: 54
# com padronização: 52 
# com nomalização: 59 

# AAPL - Apos Serie Estacionaria  log
# sem padronização: 75 com todas as features 75
# com padronização: 70 removendo os outliers: 68  | com todas as features 78
# com nomalização: 75  com todas as features 73  

### XGBOOST

In [ ]:
# fitando o modelo
xgb = XGBClassifier(random_state=42,max_depth=5)
xgb.fit(x_train, y_train)
y1_pred = xgb.predict(x_test)
print('\nClassification Report\n', metrics.classification_report(y_test, y1_pred)) 
# sem padronização: 0.56
# com padronização: 0.55
# com nomalização: 0.56

# EURUSD - Após Serie Estacionaria log
# sem padronização: 75
# com padronização: 76
# com nomalização: 75 - med 510: 73

# AAPL
# sem padronização: 53
# com padronização: 56 
# com nomalização: 58 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 79 com todas as features 77
# com padronização: 75 removendo os outliers: 74  | com todas as features 75
# com nomalização: 73  com todas as features 75  

### LGBM

In [ ]:
from sklearn.metrics import accuracy_score
from datetime import datetime

train_data=lgb.Dataset(x_train,label=y_train)

#setting parameters for lightgbm

param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}

param['metric'] = ['auc', 'binary_logloss']

#Here we have set max_depth in xgb and LightGBM to 7 to have a fair comparison between the two.

#training our model using light gbm

num_round=50

start=datetime.now()

lgbm=lgb.train(param,train_data,num_round)

stop=datetime.now()

#Execution time of the model

execution_time_lgbm = stop-start

execution_time_lgbm

#predicting on test set

ypred2=lgbm.predict(x_test)

ypred2[0:5]  # showing first 5 predictions

#converting probabilities into 0 or 1

for i in range(0,len(ypred2)):

    if ypred2[i]>=.5:       # setting threshold to .5

       ypred2[i]=1

    else:  

       ypred2[i]=0

#calculating accuracy

accuracy_lgbm = accuracy_score(ypred2,y_test)



y_test.value_counts()


#calculating roc_auc_score for xgboost
#auc_xgb =  roc_auc_score(y_test,ypred)
#auc_xgb

#calculating roc_auc_score for light gbm. 
auc_lgbm = roc_auc_score(y_test,ypred2)
auc_lgbm, accuracy_lgbm
#auc_lgbm comparison_dict = {'accuracy score':(accuracy_lgbm,accuracy_xgb),'auc score':(auc_lgbm,auc_xgb),'execution time':(execution_time_lgbm,execution_time_xgb)}

#Creating a dataframe 'comparison_df' for comparing the performance of Lightgbm and xgb. 
#comparison_df = DataFrame(comparison_dict) 
#comparison_df.index= ['LightGBM','xgboost'] 
#comparison_dfelow properly.
#Performance comparison
print('\nClassification Report\n', metrics.classification_report(y_test, ypred2)) 

# sem padronização: 0.56
# com padronização: 0.56
# com nomalização: 0.56

# EURUSD - Após Serie Estacionaria log
# sem padronização: --
# com padronização: --
# com nomalização: -- - med 510: 76

# AAPL
# sem padronização: --
# com padronização: -- 
# com nomalização: 58 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 78 com todas as features 78
# com padronização: 75 removendo os outliers: 72  | com todas as features 76
# com nomalização: 77 com todas as features 75   

### Support Vector Machine SVM

In [ ]:
#importar o modelo SVM


# instanciacao e determinacao dos hiperparâmetros do SVM: tipo de kernel
classifier = SVC(kernel='rbf')

# treinando o SVM
classifier.fit(x_train, y_train)

# aplicando o modelo treinado para a previsão da temperatura 
#em todo o conjunto de teste
predictions2_labels = classifier.predict(x_test)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
p = pd.DataFrame({'Real': y_test, 'Previsto': predictions2_labels})  

print('\nClassification Report\n', metrics.classification_report(y_test, predictions2_labels)) 
# sem padronização: 0.52
# com padronização: 0.58
# com nomalização: 0.58

# EURUSD - Após Serie Estacionaria log
# sem padronização: 75
# com padronização: 76
# com nomalização: 75 - med 510: 75

# AAPL
# sem padronização: 51
# com padronização: 55 
# com nomalização: 58 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 73 com todas as features 51
# com padronização: 75 removendo os outliers: 68 | com todas as features 78
# com nomalização: 75 com todas as features 71   

### KNN

# REGRESSAO

In [ ]:
#treinamento usando regressão linear
lr = linear_model.LinearRegression()
lr.fit(x_train, y_train)
pred= lr.predict(x_test)
cd =r2_score(y_test, pred)

f'Coeficiente de determinação (R2):{cd * 100:.2f}'

# sem padronização: 0.69
# com padronização: 
# com nomalização: 

# EURUSD - Após Serie Estacionaria log
# sem padronização: 33.03
# com padronização: 31.79
# com nomalização: 30.55 - med 510: 31,13

# AAPL
# sem padronização: -1.43
# com padronização: 
# com nomalização:  com todas as features 24.10

# AAPL - Após Serie Estacionaria  log
# sem padronização: 36.48 com todas as features 30.49
# com padronização: 28.60 com todas as features 35.62
# com nomalização: 31.82    removendo os outliers: 23.47  | com todas as features 24.10

In [ ]:
# instanciando o modelo Support Vector Regression
svr = SVR()

# treinando o modelo
svr.fit(x_train, y_train)

# prevendo os valores
y_predict = svr.predict(x_test)
cd_svr = r2_score(y_test, y_predict)

f'Coeficiente de determinação (R2):{cd_svr * 100:.2f}'
# sem padronização: -17.80
# com padronização: 
# com nomalização: 

# EURUSD - Após Serie Estacionaria log
# sem padronização: 26.34
# com padronização: 28.46
# com nomalização: 27.83 - med 510: 28,71

# AAPL
# sem padronização: -31.59
# com padronização: 
# com nomalização: 

# AAPL - Após Serie Estacionaria  log
# sem padronização: 29.70 com todas as features -24.21
# com padronização: 22.55 com todas as features 30.94
# com nomalização: 30.88    removendo os outliers: 19.34 |  com todas as features 16.92

In [ ]:
# instanciando o modelo Random Forest Regressor
rfr = RandomForestRegressor(n_estimators= 150, random_state=42).fit(x_train, y_train); #n_estimators= 10, random_state=42
pred = rfr.predict(x_test)
cd_RL = r2_score(y_test, pred)

f'Coeficiente de determinação (R2):{cd_RL * 100:.2f}'
# sem padronização: 2.49
# com padronização: 
# com nomalização: 

# EURUSD - Após Serie Estacionaria log
# sem padronização: 34.63
# com padronização: 32.81
# com nomalização: 31.88 - med 510: 32,10

# AAPL
# sem padronização: 0.27
# com padronização: 
# com nomalização:  

# AAPL - Após Serie Estacionaria log
# sem padronização: 40.88 com todas as features 35.71
# com padronização: 28.63 com todas as features 37.57
# com nomalização: 33.95    removendo os outliers: 23.35 | com todas as features 27.52

### ENSAMBLE

In [40]:
# Caso não tenha efetuado a normalização/padronização anteriormente
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(x_train)
#X_scaled_test = scaler.transform(x_test)

X_scaled = (x_train)
X_scaled_test = (x_test)

model_sgd = SGDClassifier()
model_svc = SVC()
model_dt = DecisionTreeClassifier()

#SEED = 42
#np.random.seed(SEED)

predict = pd.DataFrame(y_test)
for model in (model_sgd, model_svc, model_dt):
  model.fit(X_scaled, y_train)
  y_pred = model.predict(X_scaled_test)
  predict[model.__class__.__name__] = y_pred

predict

target_fut  SGDClassifier  SVC  \
Time                                                        
2021-03-03 20:30:00+00:00         0.0            0.0  0.0   
2021-12-02 15:30:00+00:00         0.0            0.0  1.0   
2021-10-21 14:30:00+00:00         0.0            1.0  1.0   
2021-03-23 13:30:00+00:00         1.0            0.0  1.0   
2021-11-23 17:30:00+00:00         1.0            1.0  1.0   
...                               ...            ...  ...   
2021-03-09 18:30:00+00:00         1.0            0.0  1.0   
2021-08-06 15:30:00+00:00         0.0            0.0  0.0   
2021-08-03 13:30:00+00:00         0.0            0.0  0.0   
2021-05-27 18:30:00+00:00         0.0            0.0  0.0   
2021-08-31 15:30:00+00:00         0.0            0.0  0.0   

                           DecisionTreeClassifier  
Time                                               
2021-03-03 20:30:00+00:00                     0.0  
2021-12-02 15:30:00+00:00                     0.0  
2021-10-21 14:30:00+00:00                     1.0  
2021-03-23 13:30:00+00:00                     1.0  
2021-11-23 17:30:00+00:00                     1.0  
...                                           ...  
2021-03-09 18:30:00+00:00                     1.0  
2021-08-06 15:30:00+00:00                     0.0  
2021-08-03 13:30:00+00:00                     0.0  
2021-05-27 18:30:00+00:00                     0.0  
2021-08-31 15:30:00+00:00                     0.0  

[436 rows x 4 columns]

In [41]:
# utilizando o classificador de votação
#from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators = [('sgd', model_sgd),('svc', model_svc),('dt', model_dt)]
)

predict2 = pd.DataFrame(y_test)
for model in (model_sgd, model_svc, model_dt, voting_clf):
  model.fit(X_scaled, y_train)
  y_pred = model.predict(X_scaled_test)
  predict2[model.__class__.__name__] = y_pred

predict2

target_fut  SGDClassifier  SVC  \
Time                                                        
2021-03-03 20:30:00+00:00         0.0            1.0  0.0   
2021-12-02 15:30:00+00:00         0.0            0.0  1.0   
2021-10-21 14:30:00+00:00         0.0            1.0  1.0   
2021-03-23 13:30:00+00:00         1.0            1.0  1.0   
2021-11-23 17:30:00+00:00         1.0            1.0  1.0   
...                               ...            ...  ...   
2021-03-09 18:30:00+00:00         1.0            1.0  1.0   
2021-08-06 15:30:00+00:00         0.0            0.0  0.0   
2021-08-03 13:30:00+00:00         0.0            0.0  0.0   
2021-05-27 18:30:00+00:00         0.0            1.0  0.0   
2021-08-31 15:30:00+00:00         0.0            0.0  0.0   

                           DecisionTreeClassifier  VotingClassifier  
Time                                                                 
2021-03-03 20:30:00+00:00                     0.0               0.0  
2021-12-02 15:30:00+00:00                     0.0               0.0  
2021-10-21 14:30:00+00:00                     1.0               1.0  
2021-03-23 13:30:00+00:00                     1.0               1.0  
2021-11-23 17:30:00+00:00                     1.0               1.0  
...                                           ...               ...  
2021-03-09 18:30:00+00:00                     1.0               1.0  
2021-08-06 15:30:00+00:00                     0.0               0.0  
2021-08-03 13:30:00+00:00                     0.0               0.0  
2021-05-27 18:30:00+00:00                     0.0               0.0  
2021-08-31 15:30:00+00:00                     0.0               0.0  

[436 rows x 5 columns]

In [42]:
# obtendo as acurácias
model = []
accuracy = []
for clf in (model_sgd, model_svc, model_dt, voting_clf):
  clf.fit(X_scaled, y_train)
  y_pred = clf.predict(X_scaled_test)
  model.append(clf.__class__.__name__)
  accuracy.append(accuracy_score(y_test,y_pred))

col = ['Acurácia']
ac = pd.DataFrame(data=accuracy, index = model,columns=col)
ac

Acurácia
SGDClassifier           0.743119
SVC                     0.722477
DecisionTreeClassifier  0.674312
VotingClassifier        0.750000

### continuação mesclada

### continuação original

In [ ]:
##-----------------------------------slider de horas-------------------------------------------- '''

In [39]:
with col2:
	hora_previsao = st.slider("Tempo Futuro da Previsão (horas)",
				  value=1,
				  min_value=1,
				  max_value=8,
				  step=1)
with col3:
        st.write("Previsão")
with col4:
        st.write("Probabilidade")


In [ ]:
##-----------------------------------VISUALIZAÇÃO DOS DADOS-------------------------------------------- '''

In [40]:
##-----------------------------------slider de horas-------------------------------------------- '''
if st.button('Aperte para Previsão'):
    df_viz = df[-600:]
    st.markdown("")

    # Bollinger Band
    st.subheader('Visualização das features exógenas')
    figBoll = go.Figure()
    figBoll.add_trace(
        go.Scatter(
            x = df_viz.index,
        y = df_viz['upper'],
        name = "Upper Band")
        )
    figBoll.add_trace(
        go.Scatter(
        x = df_viz.index,
        y = df_viz['mid'],
        name = "Média Móvel")
        )
    figBoll.add_trace(
        go.Scatter(
            x = df_viz.index,
        y = df_viz['low'],
        name = "Lower Band")
        )
    figBoll.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0
        ))
    figBoll.update_layout(title_text="Bollinger Band")
    figBoll.update_yaxes(tickprefix="$")
    st.plotly_chart(figBoll, use_container_width=False)

    # Gráfico RSI
    fig = px.line(df_viz, x=df_viz.index, y="rsi")
    fig.update_layout(title_text="Variação de RSI")
    st.plotly_chart(fig, use_container_width=False)
    ##-----------------------------------CRIANDO DATASET-------------------------------------------- '''

    df = target(df)
    df.dropna(inplace=True)
    df = df[['target', 'Adj Close', 'Volume', 'rsi', 'bbp', 'suport_resistencia', 'corr_class', 'media_movel', 'dia_semana', 'horario', 'mes']]
    df = constroi_features_defasadas(df,['Adj Close'],20)
    df = constroi_features_futuras(df,'target',hora_previsao)
    df_model = df.drop('target', axis=1)

    ###-----------------------------------MODELO--------------------------------------------
    if ticker == 'AAPL':
        picklefile = open("stock_pred_AAPL.pkl", "rb")
        model = pickle.load(picklefile)
    if ticker == 'AMZN':
        picklefile = open("stock_pred_AMZN.pkl", "rb")
        model = pickle.load(picklefile)

    df = df_model[-600:]
    #st.dataframe(df)
    X_test = df.drop('target_fut', axis=1)[-1:]
    #st.dataframe(X_test)
    #X_train = df[:-1].dropna().drop('target_fut', axis=1)
    #y_train = df[:-1].dropna()['target_fut']

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    y_proba = y_proba[:, 1]
    #y_pred, y_proba = modelo(df, 'target_fut')

    ###-------------------------------------------------------------------------------------
    with col3:
        #st.write("Previsão")
        if y_proba >= 0.7:
            st.info('Subir ⬆️')
        elif y_proba <= 0.3:
            st.error('Descer ⬇️')
        else:
            st.warning('Na mesma 😐')

    with col4:
        #st.write("Probabilidade")
        #st.subheader(y_proba[0])
        if y_proba >= 0.7:
            st.info(round(y_proba[0],4))
        elif y_proba <= 0.3:
            st.error(round(y_proba[0],4))
        else:
            st.warning(round(y_proba[0],4))